In [ ]:
#@title Importing the libraries
import os
import re
import pandas as pd
import librosa
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,classification_report
from scipy.fft import fft
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader,random_split,TensorDataset


# Importing the dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arvindsharma126","key":"03e2df86565eb6c7ec809398ad7ff37c"}'}

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio


100% 429M/429M [00:12<00:00, 40.3MB/s]
100% 429M/429M [00:12<00:00, 37.4MB/s]


In [ ]:
!unzip ravdess-emotional-speech-audio.zip -d ravdess-emotional-speech-audio

# Converting the audio files to a csv file

In [ ]:
path = "/content/ravdess-emotional-speech-audio"
final = np.zeros((1440,))
k = 0
for i in range(1,25):
  os.chdir(path+f"/Actor_{i:02}")
  for f in os.listdir():
    y,sr = librosa.load(f,duration=2.9)
    final[k] = len(y)
    k+=1
    # y = librosa.feature.mfcc(y = y,sr = sr,n_mfcc = 10000)
    # final = np.vstack([final,y.reshape((1,-1))])
np.std(final)

In [ ]:
final

array([63945., 63945., 63945., ..., 63945., 63945., 63945.])

In [ ]:
df = pd.DataFrame(columns=["Modality","Vocal channel","Emotion","Emotional intensity","Statement","Repetition","Actor"])
j = 0
final = []
audio = np.zeros((63945,))
path = "/content/ravdess-emotional-speech-audio"
for i in range(1,25):
  os.chdir(path+f"/Actor_{i:02}")
  for f in os.listdir():
    data = f.split(".")[0]
    data = data.split("-")
    final.append(data)
    y,sr = librosa.load(f,duration=2.9)
    audio = np.vstack([audio,y.reshape((1,-1))])
audio = audio[1:,:]

In [ ]:
final

[['03', '01', '05', '02', '02', '01', '01'],
 ['03', '01', '04', '02', '02', '01', '01'],
 ['03', '01', '01', '01', '02', '02', '01'],
 ['03', '01', '02', '02', '02', '01', '01'],
 ['03', '01', '04', '02', '02', '02', '01'],
 ['03', '01', '03', '02', '01', '02', '01'],
 ['03', '01', '08', '02', '01', '01', '01'],
 ['03', '01', '06', '01', '01', '01', '01'],
 ['03', '01', '03', '02', '02', '01', '01'],
 ['03', '01', '07', '02', '02', '02', '01'],
 ['03', '01', '08', '01', '02', '02', '01'],
 ['03', '01', '08', '02', '02', '01', '01'],
 ['03', '01', '07', '01', '01', '02', '01'],
 ['03', '01', '03', '02', '02', '02', '01'],
 ['03', '01', '04', '01', '01', '01', '01'],
 ['03', '01', '01', '01', '02', '01', '01'],
 ['03', '01', '05', '02', '02', '02', '01'],
 ['03', '01', '06', '01', '01', '02', '01'],
 ['03', '01', '06', '02', '01', '02', '01'],
 ['03', '01', '04', '01', '01', '02', '01'],
 ['03', '01', '07', '02', '01', '01', '01'],
 ['03', '01', '04', '01', '02', '02', '01'],
 ['03', '0

In [ ]:
len(final)

1440

In [ ]:
final = np.array(final).astype(int)

In [ ]:
final

array([[ 3,  1,  5, ...,  2,  1,  1],
       [ 3,  1,  4, ...,  2,  1,  1],
       [ 3,  1,  1, ...,  2,  2,  1],
       ...,
       [ 3,  1,  2, ...,  1,  2, 24],
       [ 3,  1,  4, ...,  1,  2, 24],
       [ 3,  1,  6, ...,  1,  1, 24]])

In [ ]:
dataset = pd.DataFrame(final,columns=["Modality","Vocal channel","Emotion","Emotional intensity","Statement","Repetition","Actor"])

In [ ]:
dataset

,Modality,Vocal channel,Emotion,Emotional intensity,Statement,Repetition,Actor
0,3,1,5,2,2,1,1
1,3,1,4,2,2,1,1
2,3,1,1,1,2,2,1
3,3,1,2,2,2,1,1
4,3,1,4,2,2,2,1
...,...,...,...,...,...,...,...
1435,3,1,8,1,1,2,24
1436,3,1,5,1,1,2,24
1437,3,1,2,2,1,2,24
1438,3,1,4,1,1,2,24


In [ ]:
audio = pd.DataFrame(audio)

In [ ]:
audio

,0,1,2,3,4,5,6,7,8,9,...,63935,63936,63937,63938,63939,63940,63941,63942,63943,63944
0,2.329042e-04,1.441041e-03,3.753820e-04,-9.898245e-04,-7.679415e-04,-4.599194e-04,-4.708872e-05,4.473904e-04,4.904005e-04,1.692135e-04,...,4.532097e-02,0.050078,0.045070,3.029229e-02,3.539371e-02,5.646525e-02,5.664071e-02,5.869206e-02,5.510687e-02,6.350432e-02
1,1.018356e-08,-1.005014e-08,9.524251e-09,-8.538231e-09,7.048115e-09,-4.995490e-09,2.328892e-09,9.916166e-10,-4.992537e-09,9.683954e-09,...,3.179810e-05,0.000044,0.000026,-1.672583e-06,9.198144e-06,2.301180e-05,-3.754093e-06,2.038091e-06,-1.250351e-06,7.715186e-07
2,2.278331e-05,3.115162e-05,4.793048e-06,-2.723841e-06,3.141930e-06,-4.905643e-06,3.262787e-05,6.475767e-05,3.161153e-05,3.185779e-05,...,3.252479e-06,0.000013,-0.000002,3.341737e-07,5.308339e-07,1.392216e-05,-4.699850e-07,-2.686204e-08,2.657069e-07,-4.201772e-07
3,2.429188e-05,2.923215e-05,8.048310e-06,2.269307e-05,4.088906e-05,5.115393e-05,4.717479e-05,2.486637e-05,4.857911e-05,4.697332e-05,...,8.280606e-03,0.008378,0.008179,7.974692e-03,7.836316e-03,7.985077e-03,8.453447e-03,8.543141e-03,7.968901e-03,7.787952e-03
4,1.813627e-13,-1.941189e-13,-1.676032e-13,-3.610815e-14,8.697801e-14,1.488941e-13,3.876199e-14,1.113698e-13,3.546329e-13,-1.602816e-13,...,-5.929249e-06,0.000005,-0.000005,9.120397e-06,2.205470e-05,5.102658e-06,2.106591e-05,7.540939e-08,6.271493e-06,1.096370e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-3.476922e-07,-0.000001,0.000004,3.806172e-05,3.835830e-07,6.750817e-06,1.863219e-05,1.346969e-05,5.569830e-06,-3.534751e-06
1436,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-2.550414e-06,0.000002,-0.000002,1.629342e-06,-1.235343e-06,8.224735e-07,-5.586875e-07,2.645753e-06,5.533508e-05,4.465951e-05
1437,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.530484e-04,0.000148,-0.000035,-2.018864e-04,-2.044283e-04,-1.777444e-04,-4.314623e-05,1.452080e-04,3.828391e-04,7.722454e-04
1438,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.580230e-04,0.000592,0.000535,3.330597e-04,1.518707e-04,1.039850e-04,7.744366e-05,1.735599e-05,-1.867246e-06,-5.283646e-06


In [ ]:
dataset = pd.concat([dataset,audio],axis = 1)

In [ ]:
dataset

,Modality,Vocal channel,Emotion,Emotional intensity,Statement,Repetition,Actor,0,1,2,...,63935,63936,63937,63938,63939,63940,63941,63942,63943,63944
0,3,1,5,2,2,1,1,2.329042e-04,1.441041e-03,3.753820e-04,...,4.532097e-02,0.050078,0.045070,3.029229e-02,3.539371e-02,5.646525e-02,5.664071e-02,5.869206e-02,5.510687e-02,6.350432e-02
1,3,1,4,2,2,1,1,1.018356e-08,-1.005014e-08,9.524251e-09,...,3.179810e-05,0.000044,0.000026,-1.672583e-06,9.198144e-06,2.301180e-05,-3.754093e-06,2.038091e-06,-1.250351e-06,7.715186e-07
2,3,1,1,1,2,2,1,2.278331e-05,3.115162e-05,4.793048e-06,...,3.252479e-06,0.000013,-0.000002,3.341737e-07,5.308339e-07,1.392216e-05,-4.699850e-07,-2.686204e-08,2.657069e-07,-4.201772e-07
3,3,1,2,2,2,1,1,2.429188e-05,2.923215e-05,8.048310e-06,...,8.280606e-03,0.008378,0.008179,7.974692e-03,7.836316e-03,7.985077e-03,8.453447e-03,8.543141e-03,7.968901e-03,7.787952e-03
4,3,1,4,2,2,2,1,1.813627e-13,-1.941189e-13,-1.676032e-13,...,-5.929249e-06,0.000005,-0.000005,9.120397e-06,2.205470e-05,5.102658e-06,2.106591e-05,7.540939e-08,6.271493e-06,1.096370e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,3,1,8,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,-3.476922e-07,-0.000001,0.000004,3.806172e-05,3.835830e-07,6.750817e-06,1.863219e-05,1.346969e-05,5.569830e-06,-3.534751e-06
1436,3,1,5,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,-2.550414e-06,0.000002,-0.000002,1.629342e-06,-1.235343e-06,8.224735e-07,-5.586875e-07,2.645753e-06,5.533508e-05,4.465951e-05
1437,3,1,2,2,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,4.530484e-04,0.000148,-0.000035,-2.018864e-04,-2.044283e-04,-1.777444e-04,-4.314623e-05,1.452080e-04,3.828391e-04,7.722454e-04
1438,3,1,4,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,8.580230e-04,0.000592,0.000535,3.330597e-04,1.518707e-04,1.039850e-04,7.744366e-05,1.735599e-05,-1.867246e-06,-5.283646e-06


In [ ]:
dataset.to_csv("/content/drive/MyDrive/PRML_Major_Project/audio_dataset.csv",index = False)

In [ ]:
dataset.iloc[:,7:] = np.abs(fft(dataset.iloc[:,7:],axis = 1))

In [ ]:
dataset.to_csv("/content/drive/MyDrive/PRML_Major_Project/audio_fft_dataset.csv")

In [ ]:
data = dataset.iloc[:,7:].values
mel = []
l = len(data)
for i in range(l):
  m = librosa.feature.melspectrogram(y = data[i,:])
  mel.append(m.reshape((1,-1))[0])
m = np.array(mel)

In [ ]:
m = pd.DataFrame(m)
dataset = pd.concat([dataset.iloc[:,0:7],m],axis = 1)

In [ ]:
dataset.to_csv("/content/drive/MyDrive/PRML_Major_Project/audio_mel_dataset.csv",index = False)

# Decision tree

## FFT Dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_fft_dataset.csv")
dataset

,Unnamed: 0,Modality,Vocal channel,Emotion,Emotional intensity,Statement,Repetition,Actor,0,1,...,63935,63936,63937,63938,63939,63940,63941,63942,63943,63944
0,0,3,1,5,2,2,1,1,0.562838,0.524753,...,0.356059,0.279785,0.296763,0.210512,0.331878,0.300090,0.318311,0.209626,0.407262,0.524753
1,1,3,1,4,2,2,1,1,0.044707,0.018661,...,0.007676,0.006029,0.004431,0.020777,0.026970,0.018854,0.006725,0.011850,0.015253,0.018661
2,2,3,1,1,1,2,2,1,0.065975,0.058365,...,0.032877,0.041382,0.016992,0.031579,0.035822,0.024165,0.023249,0.021512,0.024077,0.058365
3,3,3,1,2,2,2,1,1,0.019496,0.071197,...,0.030344,0.067316,0.020739,0.035819,0.065218,0.026406,0.055942,0.050371,0.066295,0.071197
4,4,3,1,4,2,2,2,1,0.065471,0.036018,...,0.012859,0.019099,0.006354,0.018731,0.023008,0.013048,0.019262,0.009203,0.044080,0.036018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,1435,3,1,8,1,1,2,24,0.009685,0.021843,...,0.006263,0.009038,0.011431,0.007916,0.042234,0.043470,0.014524,0.016566,0.005730,0.021843
1436,1436,3,1,5,1,1,2,24,0.001113,0.006725,...,0.031690,0.049714,0.025524,0.032055,0.029004,0.035755,0.030565,0.017503,0.013667,0.006725
1437,1437,3,1,2,2,1,2,24,0.196957,0.110510,...,0.058543,0.031835,0.085497,0.108549,0.029150,0.055502,0.083451,0.081203,0.010991,0.110510
1438,1438,3,1,4,1,1,2,24,0.080306,0.058616,...,0.066964,0.085966,0.049558,0.047485,0.017968,0.039558,0.026743,0.008940,0.017256,0.058616


Applying PCA and LDA

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:
pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)

Splitting the dataset

In [ ]:
train_X_pca,tv_X_pca,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_pca,valid_X_pca,test_Y,valid_Y = train_test_split(tv_X_pca,tv_Y,random_state = 12345,train_size = 0.5)
train_X_lda,tv_X_lda,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_lda,valid_X_lda,test_Y,valid_Y = train_test_split(tv_X_lda,tv_Y,random_state = 12345,train_size = 0.5)


In [ ]:
d_gi_pca,ss_gi_pca = best_hp('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
d_en_pca,ss_en_pca = best_hp('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
d_ll_pca,ss_ll_pca = best_hp('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)

d_gi_lda,ss_gi_lda = best_hp('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
d_en_lda,ss_en_lda = best_hp('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
d_ll_lda,ss_ll_lda = best_hp('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

In [ ]:
tree_gi_pca = DecisionTreeClassifier(criterion='gini',max_depth=d_gi_pca,min_samples_split=ss_gi_pca).fit(train_X_pca,train_Y)
tree_en_pca = DecisionTreeClassifier(criterion='entropy',max_depth=d_en_pca,min_samples_split=ss_en_pca).fit(train_X_pca,train_Y)
tree_ll_pca = DecisionTreeClassifier(criterion='log_loss',max_depth=d_ll_pca,min_samples_split=ss_ll_pca).fit(train_X_pca,train_Y)

tree_gi_lda = DecisionTreeClassifier(criterion='gini',max_depth=d_gi_lda,min_samples_split=ss_gi_lda).fit(train_X_lda,train_Y)
tree_en_lda = DecisionTreeClassifier(criterion='entropy',max_depth=d_en_lda,min_samples_split=ss_en_lda).fit(train_X_lda,train_Y)
tree_ll_lda = DecisionTreeClassifier(criterion='log_loss',max_depth=d_ll_lda,min_samples_split=ss_ll_lda).fit(train_X_lda,train_Y)



Results

In [ ]:
clk = [tree_gi_pca,tree_en_pca,tree_ll_pca,tree_gi_lda,tree_en_lda,tree_ll_lda]
clk_name = ['tree_gi_pca','tree_en_pca','tree_ll_pca','tree_gi_lda','tree_en_lda','tree_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

tree_gi_pca
              precision    recall  f1-score   support

           1       0.06      0.14      0.08        14
           2       0.33      0.35      0.34        34
           3       0.27      0.19      0.22        42
           4       0.23      0.19      0.21        43
           5       0.43      0.28      0.34        32
           6       0.34      0.31      0.32        39
           7       0.31      0.38      0.34        37
           8       0.35      0.38      0.37        47

    accuracy                           0.29       288
   macro avg       0.29      0.28      0.28       288
weighted avg       0.31      0.29      0.29       288

tree_en_pca
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        14
           2       0.33      0.76      0.46        34
           3       0.20      0.83      0.32        42
           4       0.00      0.00      0.00        43
           5       0.48      0.50      0.49        32


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

## Melspectogram Dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_mel_dataset.csv")
dataset

,Modality,Vocal channel,Emotion,Emotional intensity,Statement,Repetition,Actor,0,1,2,...,15990,15991,15992,15993,15994,15995,15996,15997,15998,15999
0,3,1,5,2,2,1,1,7.569644e-07,5.010802e-06,8.384924e-06,...,5.499699e-05,5.885491e-06,1.657120e-05,7.602131e-05,6.069553e-05,2.774049e-05,1.739454e-05,3.164505e-05,3.757783e-05,7.998535e-05
1,3,1,4,2,2,1,1,2.794822e-10,1.450041e-10,1.175133e-10,...,9.339636e-10,1.220944e-09,3.162984e-09,2.801177e-09,1.114704e-09,4.183812e-10,3.350986e-10,3.824072e-10,3.206519e-10,2.361991e-10
2,3,1,1,1,2,2,1,1.332402e-07,1.478175e-07,1.084568e-07,...,1.416472e-10,7.174033e-11,1.259040e-10,1.607572e-10,3.343664e-11,5.016731e-11,4.086213e-11,3.167913e-11,1.258924e-10,2.130951e-10
3,3,1,2,2,2,1,1,1.523058e-06,1.167175e-06,2.473346e-07,...,3.866556e-08,9.828029e-08,1.197246e-07,8.062022e-08,3.255798e-08,2.372752e-09,6.229968e-09,3.475527e-08,3.888261e-08,5.073903e-07
4,3,1,4,2,2,2,1,1.922096e-10,3.086805e-10,1.442073e-10,...,6.283513e-09,3.751002e-09,1.515500e-09,1.571249e-09,1.424604e-09,1.195403e-09,5.786857e-10,5.123155e-10,4.680149e-10,3.469635e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,3,1,8,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,9.985296e-10,4.968911e-10,3.221174e-10,3.396013e-10,3.091778e-10,3.297025e-10,4.210789e-10,4.688974e-10,7.026459e-10,5.663512e-10
1436,3,1,5,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,6.965746e-08,6.391710e-08,5.344007e-08,5.219970e-08,1.727351e-08,7.570125e-09,3.529409e-09,1.575603e-09,1.517090e-09,1.485708e-09
1437,3,1,2,2,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,2.448344e-08,8.441065e-09,8.191249e-08,2.261750e-07,1.162307e-07,5.961151e-08,5.394006e-08,5.558606e-08,3.873275e-08,4.627699e-08
1438,3,1,4,1,1,2,24,0.000000e+00,0.000000e+00,0.000000e+00,...,6.350594e-08,1.018883e-07,1.052788e-07,5.846529e-08,3.531843e-08,2.501399e-08,2.186149e-08,2.132620e-08,1.741378e-08,1.488148e-08


Applying PCA and LDA

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:
pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)

In [ ]:
train_X_pca,tv_X_pca,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_pca,valid_X_pca,test_Y,valid_Y = train_test_split(tv_X_pca,tv_Y,random_state = 12345,train_size = 0.5)
train_X_lda,tv_X_lda,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_lda,valid_X_lda,test_Y,valid_Y = train_test_split(tv_X_lda,tv_Y,random_state = 12345,train_size = 0.5)


In [ ]:
d_gi_pca,ss_gi_pca = best_hp('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
d_en_pca,ss_en_pca = best_hp('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
d_ll_pca,ss_ll_pca = best_hp('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)

d_gi_lda,ss_gi_lda = best_hp('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
d_en_lda,ss_en_lda = best_hp('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
d_ll_lda,ss_ll_lda = best_hp('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

In [ ]:
tree_gi_pca = DecisionTreeClassifier(criterion='gini',max_depth=d_gi_pca,min_samples_split=ss_gi_pca).fit(train_X_pca,train_Y)
tree_en_pca = DecisionTreeClassifier(criterion='entropy',max_depth=d_en_pca,min_samples_split=ss_en_pca).fit(train_X_pca,train_Y)
tree_ll_pca = DecisionTreeClassifier(criterion='log_loss',max_depth=d_ll_pca,min_samples_split=ss_ll_pca).fit(train_X_pca,train_Y)

tree_gi_lda = DecisionTreeClassifier(criterion='gini',max_depth=d_gi_lda,min_samples_split=ss_gi_lda).fit(train_X_lda,train_Y)
tree_en_lda = DecisionTreeClassifier(criterion='entropy',max_depth=d_en_lda,min_samples_split=ss_en_lda).fit(train_X_lda,train_Y)
tree_ll_lda = DecisionTreeClassifier(criterion='log_loss',max_depth=d_ll_lda,min_samples_split=ss_ll_lda).fit(train_X_lda,train_Y)



Result

In [ ]:
clk = [tree_gi_pca,tree_en_pca,tree_ll_pca,tree_gi_lda,tree_en_lda,tree_ll_lda]
clk_name = ['tree_gi_pca','tree_en_pca','tree_ll_pca','tree_gi_lda','tree_en_lda','tree_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

tree_gi_pca
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        14
           2       0.34      0.68      0.46        34
           3       0.00      0.00      0.00        42
           4       0.23      0.26      0.24        43
           5       0.48      0.41      0.44        32
           6       0.00      0.00      0.00        39
           7       0.20      0.38      0.26        37
           8       0.29      0.47      0.35        47

    accuracy                           0.29       288
   macro avg       0.19      0.27      0.22       288
weighted avg       0.20      0.29      0.23       288

tree_en_pca
              precision    recall  f1-score   support

           1       0.44      0.50      0.47        14
           2       0.28      0.65      0.39        34
           3       0.10      0.02      0.04        42
           4       0.00      0.00      0.00        43
           5       0.30      0.41      0.35        32


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

# Adaboost

## FFT dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_fft_dataset.csv")

applying PCA and LDA

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:

pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)

In [ ]:
train_X_pca,tv_X_pca,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_pca,valid_X_pca,test_Y,valid_Y = train_test_split(tv_X_pca,tv_Y,random_state = 12345,train_size = 0.5)
train_X_lda,tv_X_lda,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_lda,valid_X_lda,test_Y,valid_Y = train_test_split(tv_X_lda,tv_Y,random_state = 12345,train_size = 0.5)


In [ ]:
kgip = best_para('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
kenp = best_para('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
kllp = best_para('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)
kgil = best_para('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
kenl = best_para('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
klll = best_para('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

Results

In [ ]:
ada_gi_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = 7),n_estimators=kgip).fit(train_X_pca,train_Y)
ada_en_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = 7),n_estimators=kenp).fit(train_X_pca,train_Y)
ada_ll_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = 7),n_estimators=kllp).fit(train_X_pca,train_Y)
ada_gi_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = 7),n_estimators=kgil).fit(train_X_lda,train_Y)
ada_en_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = 7),n_estimators=kenl).fit(train_X_lda,train_Y)
ada_ll_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = 7),n_estimators=klll).fit(train_X_lda,train_Y)


clk = [ada_gi_pca,ada_en_pca,ada_ll_pca,ada_gi_lda,ada_en_lda,ada_ll_lda]
clk_name = ['ada_gi_pca','ada_en_pca','ada_ll_pca','ada_gi_lda','ada_en_lda','ada_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

ada_gi_pca
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        14
           2       0.34      0.62      0.44        34
           3       0.19      0.17      0.18        42
           4       0.32      0.28      0.30        43
           5       0.33      0.34      0.34        32
           6       0.25      0.18      0.21        39
           7       0.12      0.19      0.15        37
           8       0.28      0.17      0.21        47

    accuracy                           0.25       288
   macro avg       0.23      0.24      0.23       288
weighted avg       0.25      0.25      0.24       288

ada_en_pca
              precision    recall  f1-score   support

           1       0.50      0.14      0.22        14
           2       0.37      0.71      0.48        34
           3       0.19      0.17      0.18        42
           4       0.20      0.09      0.13        43
           5       0.35      0.47      0.40        32
  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
kgip,dgip,sgip = best_para_with_hp('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
kenp,denp,senp = best_para_with_hp('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
kllp,dllp,sllp = best_para_with_hp('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)
kgil,dgil,sgil = best_para_with_hp('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
kenl,denl,senl = best_para_with_hp('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
klll,dlll,slll = best_para_with_hp('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

Results

In [ ]:
ada_gi_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = dgip,min_samples_split=sgip),n_estimators=kgip).fit(train_X_pca,train_Y)
ada_en_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = denp,min_samples_split=senp),n_estimators=kenp).fit(train_X_pca,train_Y)
ada_ll_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = dllp,min_samples_split=sllp),n_estimators=kllp).fit(train_X_pca,train_Y)
ada_gi_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = dgil,min_samples_split=sgil),n_estimators=kgil).fit(train_X_lda,train_Y)
ada_en_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = denl,min_samples_split=senl),n_estimators=kenl).fit(train_X_lda,train_Y)
ada_ll_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = dlll,min_samples_split=slll),n_estimators=klll).fit(train_X_lda,train_Y)


clk = [ada_gi_pca,ada_en_pca,ada_ll_pca,ada_gi_lda,ada_en_lda,ada_ll_lda]
clk_name = ['ada_gi_pca','ada_en_pca','ada_ll_pca','ada_gi_lda','ada_en_lda','ada_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

ada_gi_pca
              precision    recall  f1-score   support

           1       0.05      0.07      0.06        14
           2       0.37      0.41      0.39        34
           3       0.21      0.21      0.21        42
           4       0.19      0.14      0.16        43
           5       0.38      0.56      0.46        32
           6       0.22      0.15      0.18        39
           7       0.17      0.24      0.20        37
           8       0.31      0.19      0.24        47

    accuracy                           0.25       288
   macro avg       0.24      0.25      0.24       288
weighted avg       0.25      0.25      0.24       288

ada_en_pca
              precision    recall  f1-score   support

           1       0.14      0.07      0.10        14
           2       0.38      0.44      0.41        34
           3       0.50      0.05      0.09        42
           4       0.24      0.40      0.30        43
           5       0.29      0.53      0.38        32
  

## Melspectogram dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_mel_dataset.csv")

applying PCA and LDA

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:
pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)

In [ ]:
train_X_pca,tv_X_pca,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_pca,valid_X_pca,test_Y,valid_Y = train_test_split(tv_X_pca,tv_Y,random_state = 12345,train_size = 0.5)
train_X_lda,tv_X_lda,train_Y,tv_Y = train_test_split(X_pca,Y,random_state = 12345,train_size = 0.6)
test_X_lda,valid_X_lda,test_Y,valid_Y = train_test_split(tv_X_lda,tv_Y,random_state = 12345,train_size = 0.5)


In [ ]:
kgip = best_para('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
kenp = best_para('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
kllp = best_para('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)
kgil = best_para('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
kenl = best_para('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
klll = best_para('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

Result

In [ ]:
ada_gi_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = 7),n_estimators=kgip).fit(train_X_pca,train_Y)
ada_en_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = 7),n_estimators=kenp).fit(train_X_pca,train_Y)
ada_ll_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = 7),n_estimators=kllp).fit(train_X_pca,train_Y)
ada_gi_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = 7),n_estimators=kgil).fit(train_X_lda,train_Y)
ada_en_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = 7),n_estimators=kenl).fit(train_X_lda,train_Y)
ada_ll_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = 7),n_estimators=klll).fit(train_X_lda,train_Y)


clk = [ada_gi_pca,ada_en_pca,ada_ll_pca,ada_gi_lda,ada_en_lda,ada_ll_lda]
clk_name = ['ada_gi_pca','ada_en_pca','ada_ll_pca','ada_gi_lda','ada_en_lda','ada_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

ada_gi_pca
              precision    recall  f1-score   support

           1       0.40      0.14      0.21        14
           2       0.36      0.53      0.43        34
           3       0.34      0.36      0.35        42
           4       0.21      0.16      0.18        43
           5       0.29      0.38      0.32        32
           6       0.24      0.21      0.22        39
           7       0.30      0.32      0.31        37
           8       0.39      0.34      0.36        47

    accuracy                           0.31       288
   macro avg       0.32      0.30      0.30       288
weighted avg       0.31      0.31      0.30       288

ada_en_pca
              precision    recall  f1-score   support

           1       0.50      0.21      0.30        14
           2       0.36      0.65      0.46        34
           3       0.19      0.21      0.20        42
           4       0.16      0.09      0.12        43
           5       0.39      0.41      0.40        32
  

In [ ]:
kgip,dgip,sgip = best_para_with_hp('gini',train_X_pca,train_Y,valid_X_pca,valid_Y)
kenp,denp,senp = best_para_with_hp('entropy',train_X_pca,train_Y,valid_X_pca,valid_Y)
kllp,dllp,sllp = best_para_with_hp('log_loss',train_X_pca,train_Y,valid_X_pca,valid_Y)
kgil,dgil,sgil = best_para_with_hp('gini',train_X_lda,train_Y,valid_X_lda,valid_Y)
kenl,denl,senl = best_para_with_hp('entropy',train_X_lda,train_Y,valid_X_lda,valid_Y)
klll,dlll,slll = best_para_with_hp('log_loss',train_X_lda,train_Y,valid_X_lda,valid_Y)

Result

In [ ]:
ada_gi_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = dgip,min_samples_split=sgip),n_estimators=kgip).fit(train_X_pca,train_Y)
ada_en_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = denp,min_samples_split=senp),n_estimators=kenp).fit(train_X_pca,train_Y)
ada_ll_pca = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = dllp,min_samples_split=sllp),n_estimators=kllp).fit(train_X_pca,train_Y)
ada_gi_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='gini',max_depth = dgil,min_samples_split=sgil),n_estimators=kgil).fit(train_X_lda,train_Y)
ada_en_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='entropy',max_depth = denl,min_samples_split=senl),n_estimators=kenl).fit(train_X_lda,train_Y)
ada_ll_lda = AdaBoostClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',max_depth = dlll,min_samples_split=slll),n_estimators=klll).fit(train_X_lda,train_Y)


clk = [ada_gi_pca,ada_en_pca,ada_ll_pca,ada_gi_lda,ada_en_lda,ada_ll_lda]
clk_name = ['ada_gi_pca','ada_en_pca','ada_ll_pca','ada_gi_lda','ada_en_lda','ada_ll_lda']
l = len(clk)
for i in range(l):
  print(clk_name[i])
  if i < 3:
    test_X = test_X_pca
  else:
    test_X = test_X_lda
  print(classification_report(test_Y,clk[i].predict(test_X)))

ada_gi_pca
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        14
           2       0.35      0.68      0.46        34
           3       0.20      0.02      0.04        42
           4       0.24      0.26      0.25        43
           5       0.44      0.47      0.45        32
           6       0.19      0.28      0.23        39
           7       0.21      0.38      0.27        37
           8       0.31      0.09      0.13        47

    accuracy                           0.27       288
   macro avg       0.24      0.27      0.23       288
weighted avg       0.26      0.27      0.24       288

ada_en_pca
              precision    recall  f1-score   support

           1       0.03      0.07      0.04        14
           2       0.36      0.47      0.41        34
           3       0.18      0.24      0.20        42
           4       0.14      0.07      0.09        43
           5       0.12      0.09      0.10        32
  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

# NN

## FFT dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_fft_dataset.csv")

Applying pca and lda

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)
X_pca = torch.tensor(X_pca)
X_lda = torch.tensor(X_lda)
Y = torch.tensor(Y.values - 1)

In [ ]:
dataset_pca = TensorDataset(X_pca,Y)
dataset_lda = TensorDataset(X_lda,Y)


In [ ]:
len(Y)

1440

Splitting the dataset

In [ ]:
train_pca,test_pca,valid_pca = random_split(dataset_pca,[864,288,288])
train_lda,test_lda,valid_lda = random_split(dataset_lda,[864,288,288])


In [ ]:
train_pca_dl = DataLoader(dataset = train_pca,batch_size = 64)
test_pca_dl = DataLoader(dataset = test_pca)
valid_pca_dl = DataLoader(dataset = valid_pca)
train_lda_dl = DataLoader(dataset = train_lda,batch_size = 64)
test_lda_dl = DataLoader(dataset = test_lda)
valid_lda_dl = DataLoader(dataset = valid_lda)

ANN with sigmoid as activation function and with three hidden layer

In [ ]:
class model_sig(nn.Module):
  def __init__(self,input_size,num_classes):
    super(model_sig,self).__init__()
    self.hid_lay1 = nn.Linear(input_size,500)
    self.act_fn1 = nn.Sigmoid()
    self.hid_lay2 = nn.Linear(500,300)
    self.act_fn2 = nn.Sigmoid()
    self.hid_lay3 = nn.Linear(300,8)
    self.act_fn3 = nn.Sigmoid()
    self.output_layer = nn.Softmax()
  
  def forward(self,x):
    x = self.hid_lay1(x)
    x = self.act_fn1(x)
    x = self.hid_lay2(x)
    x = self.act_fn2(x)
    x = self.hid_lay3(x)
    x = self.act_fn3(x)
    x = self.output_layer(x)
    return x
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Using PCA dataset

In [ ]:
model = model_sig(500,8).to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr = 0.0001)

In [ ]:
best_acc = 0.0

saving the best model on validation set

In [ ]:
for epoch in range(1000):

    running_loss = 0.0
    acc = 0.0
    j = 0
    for i, data in enumerate(train_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
    if epoch % 25 == 24:
        print(f'[{epoch + 1}] Training : loss: {running_loss:0.2f} accuracy : {acc/j}',end='')
    with torch.no_grad():
      acc = 0.0
      j = 0
      running_loss = 0.0
      for i, data in enumerate(valid_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
      if best_acc < acc/j:
        torch.save(model,'/content/drive/MyDrive/PRML_Major_Project/model_pca.pth')
        best_acc = acc/j
    if epoch % 25 == 24:
        print(f' Validation :  loss: {running_loss:0.2f} accuracy : {acc/j}')
print('Finished Training')

<ipython-input-12-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


[25] Training : loss: 1674.02 accuracy : 0.9710648148148148 Validation :  loss: 592.35 accuracy : 0.3263888888888889
[50] Training : loss: 1674.02 accuracy : 0.9629629629629629 Validation :  loss: 592.18 accuracy : 0.3055555555555556
[75] Training : loss: 1673.87 accuracy : 0.9398148148148148 Validation :  loss: 591.67 accuracy : 0.3055555555555556
[100] Training : loss: 1673.87 accuracy : 0.9317129629629629 Validation :  loss: 591.43 accuracy : 0.2881944444444444
[125] Training : loss: 1673.87 accuracy : 0.9305555555555556 Validation :  loss: 590.92 accuracy : 0.2986111111111111
[150] Training : loss: 1674.02 accuracy : 0.9652777777777778 Validation :  loss: 593.03 accuracy : 0.2673611111111111
[175] Training : loss: 1673.87 accuracy : 0.9490740740740741 Validation :  loss: 592.42 accuracy : 0.2847222222222222
[200] Training : loss: 1673.87 accuracy : 0.9363425925925926 Validation :  loss: 592.39 accuracy : 0.3055555555555556
[225] Training : loss: 1673.87 accuracy : 0.931712962962962

In [ ]:
model =torch.load('/content/drive/MyDrive/PRML_Major_Project/model_pca.pth')

result

In [ ]:
with torch.no_grad():
    acc = 0.0
    j = 0
    for i, data in enumerate(test_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        opt.zero_grad()
        outputs = model(inputs)
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.numpy()
        acc += np.sum(pred == labels)
        j += 1
print(f"accuracy : {acc/j}")

accuracy : 0.3819444444444444


<ipython-input-12-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


### using LDA dataset

In [ ]:
model = model_sig(7,8).to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr = 0.0001)

In [ ]:
best_acc = 0.0

In [ ]:
for epoch in range(1000):

    running_loss = 0.0
    acc = 0.0
    j = 0
    for i, data in enumerate(train_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
    if epoch % 25 == 24:
        print(f'[{epoch + 1}] Training : loss: {running_loss:0.2f} accuracy : {acc/j}',end='')
    with torch.no_grad():
      acc = 0.0
      j = 0
      running_loss = 0.0
      for i, data in enumerate(valid_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
      if best_acc < acc/j:
        torch.save(model,'/content/drive/MyDrive/PRML_Major_Project/model_lda.pth')
        best_acc = acc/j
    if epoch % 25 == 24:
        print(f' Validation :  loss: {running_loss:0.2f} accuracy : {acc/j}')
print('Finished Training')

<ipython-input-12-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


[25] Training : loss: 1677.31 accuracy : 0.9120370370370371 Validation :  loss: 560.36 accuracy : 0.8854166666666666
[50] Training : loss: 1676.44 accuracy : 0.9166666666666666 Validation :  loss: 560.31 accuracy : 0.8819444444444444
[75] Training : loss: 1675.98 accuracy : 0.9166666666666666 Validation :  loss: 560.27 accuracy : 0.8819444444444444
[100] Training : loss: 1675.82 accuracy : 0.9166666666666666 Validation :  loss: 560.28 accuracy : 0.8854166666666666
[125] Training : loss: 1675.78 accuracy : 0.9166666666666666 Validation :  loss: 560.27 accuracy : 0.8854166666666666
[150] Training : loss: 1675.76 accuracy : 0.9189814814814815 Validation :  loss: 560.27 accuracy : 0.8854166666666666
[175] Training : loss: 1675.76 accuracy : 0.9398148148148148 Validation :  loss: 560.28 accuracy : 0.9201388888888888
[200] Training : loss: 1675.64 accuracy : 0.9340277777777778 Validation :  loss: 560.47 accuracy : 0.9201388888888888
[225] Training : loss: 1675.59 accuracy : 0.928240740740740

In [ ]:
model =torch.load('/content/drive/MyDrive/PRML_Major_Project/model_lda.pth')

Result

In [ ]:
with torch.no_grad():
    acc = 0.0
    j = 0
    for i, data in enumerate(test_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        opt.zero_grad()
        outputs = model(inputs)
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.numpy()
        acc += np.sum(pred == labels)
        j += 1
print(f"accuracy : {acc/j}")

accuracy : 0.9479166666666666


<ipython-input-12-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


## Malspectogram dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/PRML_Major_Project/audio_mel_dataset.csv")

applying pca and lda

In [ ]:
pca = PCA(n_components = 500)
lda = LDA(n_components = 7)

In [ ]:
Y = dataset["Emotion"]
X = dataset.iloc[:,7:]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pca = pca.fit(X)
lda = lda.fit(X,Y)
X_pca = pca.transform(X)
X_lda = lda.transform(X)
X_pca = torch.tensor(X_pca)
X_lda = torch.tensor(X_lda)
Y = torch.tensor(Y.values - 1)

In [ ]:
dataset_pca = TensorDataset(X_pca,Y)
dataset_lda = TensorDataset(X_lda,Y)


In [ ]:
len(Y)

1440

In [ ]:
train_pca,test_pca,valid_pca = random_split(dataset_pca,[864,288,288])
train_lda,test_lda,valid_lda = random_split(dataset_lda,[864,288,288])


In [ ]:
train_pca_dl = DataLoader(dataset = train_pca,batch_size = 64)
test_pca_dl = DataLoader(dataset = test_pca)
valid_pca_dl = DataLoader(dataset = valid_pca)
train_lda_dl = DataLoader(dataset = train_lda,batch_size = 64)
test_lda_dl = DataLoader(dataset = test_lda)
valid_lda_dl = DataLoader(dataset = valid_lda)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### using pca dataset


In [ ]:
model = model_sig(500,8).to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr = 0.0001)

In [ ]:
best_acc = 0.0

In [ ]:
for epoch in range(1000):

    running_loss = 0.0
    acc = 0.0
    j = 0
    for i, data in enumerate(train_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
    if epoch % 25 == 24:
        print(f'[{epoch + 1}] Training : loss: {running_loss:0.2f} accuracy : {acc/j}',end='')
    with torch.no_grad():
      acc = 0.0
      j = 0
      running_loss = 0.0
      for i, data in enumerate(valid_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
      if best_acc < acc/j:
        torch.save(model,'/content/drive/MyDrive/PRML_Major_Project/model2_pca.pth')
        best_acc = acc/j
    if epoch % 25 == 24:
        print(f' Validation :  loss: {running_loss:0.2f} accuracy : {acc/j}')
print('Finished Training')

<ipython-input-53-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


[25] Training : loss: 1712.20 accuracy : 0.6875 Validation :  loss: 591.05 accuracy : 0.2916666666666667
[50] Training : loss: 1703.53 accuracy : 0.7349537037037037 Validation :  loss: 591.77 accuracy : 0.2847222222222222
[75] Training : loss: 1702.37 accuracy : 0.7395833333333334 Validation :  loss: 590.61 accuracy : 0.3055555555555556
[100] Training : loss: 1701.08 accuracy : 0.7442129629629629 Validation :  loss: 592.61 accuracy : 0.2777777777777778
[125] Training : loss: 1700.95 accuracy : 0.7430555555555556 Validation :  loss: 592.62 accuracy : 0.28125
[150] Training : loss: 1701.00 accuracy : 0.7453703703703703 Validation :  loss: 592.02 accuracy : 0.28125
[175] Training : loss: 1701.04 accuracy : 0.7465277777777778 Validation :  loss: 591.76 accuracy : 0.2951388888888889
[200] Training : loss: 1700.42 accuracy : 0.7488425925925926 Validation :  loss: 591.95 accuracy : 0.2951388888888889
[225] Training : loss: 1700.39 accuracy : 0.7465277777777778 Validation :  loss: 592.22 accur

In [ ]:
model =torch.load('/content/drive/MyDrive/PRML_Major_Project/model2_pca.pth')

Results

In [ ]:
with torch.no_grad():
    acc = 0.0
    j = 0
    for i, data in enumerate(test_pca):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        opt.zero_grad()
        outputs = model(inputs)
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.numpy()
        acc += np.sum(pred == labels)
        j += 1
print(f"accuracy : {acc/j}")

accuracy : 0.2951388888888889


<ipython-input-53-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


### using lda dataset


In [ ]:
model = model_sig(7,8).to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr = 0.0001)

In [ ]:
best_acc = 0.0

In [ ]:
for epoch in range(1000):

    running_loss = 0.0
    acc = 0.0
    j = 0
    for i, data in enumerate(train_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
    if epoch % 25 == 24:
        print(f'[{epoch + 1}] Training : loss: {running_loss:0.2f} accuracy : {acc/j}',end='')
    with torch.no_grad():
      acc = 0.0
      j = 0
      running_loss = 0.0
      for i, data in enumerate(valid_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        labels = labels.to(device = device)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        acc += np.sum(pred == labels)
        j += 1
      if best_acc < acc/j:
        torch.save(model,'/content/drive/MyDrive/PRML_Major_Project/model2_lda.pth')
        best_acc = acc/j
    if epoch % 25 == 24:
        print(f' Validation :  loss: {running_loss:0.2f} accuracy : {acc/j}')
print('Finished Training')

<ipython-input-53-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)


[25] Training : loss: 1665.34 accuracy : 0.9942129629629629 Validation :  loss: 555.36 accuracy : 0.9895833333333334
[50] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.44 accuracy : 0.9861111111111112
[75] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.40 accuracy : 0.9895833333333334
[100] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.41 accuracy : 0.9895833333333334
[125] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.41 accuracy : 0.9895833333333334
[150] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.41 accuracy : 0.9895833333333334
[175] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.42 accuracy : 0.9895833333333334
[200] Training : loss: 1665.28 accuracy : 0.9942129629629629 Validation :  loss: 555.42 accuracy : 0.9895833333333334
[225] Training : loss: 1665.28 accuracy : 0.994212962962962

In [ ]:
model =torch.load('/content/drive/MyDrive/PRML_Major_Project/model2_lda.pth')

Results

In [ ]:
with torch.no_grad():
    acc = 0.0
    j = 0
    for i, data in enumerate(test_lda):
        inputs, labels = data
        inputs = inputs.to(device = device).float()
        opt.zero_grad()
        outputs = model(inputs)
        _,pred = torch.max(outputs,0)
        pred = pred.cpu().numpy()
        labels = labels.numpy()
        acc += np.sum(pred == labels)
        j += 1
print(f"accuracy : {acc/j}")

accuracy : 0.9930555555555556


<ipython-input-53-92a50441495b>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.output_layer(x)
